In [ ]:
import neurokit2 as nk
from tqdm import tqdm
import pandas as pd
import numpy as np

def calculate_hrv_grouped(df, groupby=['fitbit_id', 'date']):
    grouped = df.groupby(groupby)
    final_df = pd.DataFrame()
    
    for group_name, group_data in tqdm(grouped, miniters=1, mininterval=1):
        if len(group_data) < 2:
            continue
        
        try:
            RR_interval = 60000 / group_data['value']
            
            peaks = nk.intervals_to_peaks(RR_interval)
            
            hrv_results = nk.hrv_time(peaks, show=False)
            
            hrv_df = pd.DataFrame(hrv_results, index=[0])
            hrv_df['fitbit_id'] = group_name[0]
            hrv_df['date'] = group_name[1]
            
            final_df = pd.concat([final_df, hrv_df], ignore_index=True)
        
        except Exception as e:
            print(f"그룹 {group_name} 처리 중 오류 발생: {str(e)}")
    
    return final_df

In [ ]:
df = pd.read_csv("heart_rates(id).csv")
del df['id']

In [ ]:
def make_date(value):
    return value.date()
    
df['timestamp'] = pd.to_datetime(df['timestamp'])

df['date'] = df['timestamp'].apply(make_date)

In [ ]:
hrv_df = calculate_hrv_grouped(df, groupby=['fitbit_id', 'date'])

In [ ]:
list(hrv_df.columns)

hrv_df = hrv_df[[ 'fitbit_id',
 'date',
 'HRV_MeanNN',
 'HRV_SDNN',
 'HRV_SDANN1',
 'HRV_SDNNI1',
 'HRV_SDANN2',
 'HRV_SDNNI2',
 'HRV_SDANN5',
 'HRV_SDNNI5',
 'HRV_RMSSD',
 'HRV_SDSD',
 'HRV_CVNN',
 'HRV_CVSD',
 'HRV_MedianNN',
 'HRV_MadNN',
 'HRV_MCVNN',
 'HRV_IQRNN',
 'HRV_SDRMSSD',
 'HRV_Prc20NN',
 'HRV_Prc80NN',
 'HRV_pNN50',
 'HRV_pNN20',
 'HRV_MinNN',
 'HRV_MaxNN',
 'HRV_HTI',
 'HRV_TINN']]

In [ ]:
hrv_df.rename(columns={'fitbit_id':'patient_id'}, inplace=True)
hrv_df.to_csv("HRV.csv", encoding="utf-8-sig", index=False)